In [1]:
# verify the library installation for Mega Detector
from utils.general import non_max_suppression, scale_coords, xyxy2xywh
from utils.augmentations import letterbox
import visualization.visualization_utils as viz_utils
import numpy as np
import ct_utils
import os
import torchvision.transforms as T
import torch
from PIL import Image
import cv2
import torch.backends.cudnn as cudnn
import torch.nn as nn
import shutil

In [2]:
# verifying the availability of the GPUs
if torch.cuda.is_available():
    device = "cuda"
    print(f"PyTorch detected: {torch.cuda.get_device_name(0)} for processing.. ")
    print(f"Numbe of GPUs found: {torch.cuda.device_count()}")
    device = torch.device("cuda") # if there are multiple GPUs assign the Device Id
    cudnn.benchmark = True
else:
    device = "cpu"

PyTorch detected: NVIDIA GeForce RTX 4090 for processing.. 
Numbe of GPUs found: 1


In [3]:
current_path = os.getcwd()
md_model_path = os.path.join(r'D:\PhD\work_space\Detectifier\v1\Detect\md_v5b.0.0.pt')
#input_dir_path = os.path.join(current_path, 'RAW', 'OOB_2')
#output_dir_path = os.path.join(current_path, 'Filter', 'OOB')

In [4]:
print(f'Loading a model onto the {device} for inference analysis.. ')
try:
    model_checkpoint = torch.load(md_model_path, device)
    model = model_checkpoint['model'].float().fuse().eval()
    print(f'[info]: Model is loaded to the {device}')
except OSError as oer:
    print(oer)

Loading a model onto the cuda for inference analysis.. 


Fusing layers... 
Model summary: 574 layers, 139990096 parameters, 0 gradients, 207.9 GFLOPs


[info]: Model is loaded to the cuda


In [5]:
for m in model.modules():
    if isinstance(m, nn.Upsample):
        m.recompute_scale_factor = None

In [6]:
# Variables used by the megadetector model
IMAGE_SIZE = 1280  # image size used in training
STRIDE = 64
DETECTION_THRESHOLD = 0.25
# Label mapping for MegaDetector
DEFAULT_DETECTOR_LABEL_MAP = ['animal', 'person', 'vehicle']

In [7]:
def MegaDetector_Filter(input_image_path, output_category_dir_path):
    """ Function to crop the animal from the images with highest confidence score """
    # print(input_image_path)
    # List variables to collect the output
    final_output = {'input_file' : f'{os.path.basename(input_image_path)}'}
    tot_detections = []
    labels = []
    bbox = []
    scores = []
    max_conf = 0.0
    # print(f'Pre-processing {os.path.basename(input_image_path)} file.. ')
    img_data = cv2.imread(input_image_path) # read the image using cv2 library
    cpy_img_data = img_data
    img_array = np.asarray(img_data) # Convert the image data into a numpy array
    img_resize = letterbox(img_array, new_shape=IMAGE_SIZE, stride=STRIDE, auto=True)[0] # Resize the image shape as per the megadetector model
    img_transpose = img_resize.transpose((2, 0, 1))  # HWC to CHW; PIL Image is RGB already
    img = np.ascontiguousarray(img_transpose)
    img = torch.from_numpy(img)
    img = img.to(device)
    img = img.float()
    img /= 255
    if len(img.shape) == 3:  # always true for now, TODO add inference using larger batch size
        img = torch.unsqueeze(img, 0)
    # print('Mega detector look for objects in the image.. ')
    with torch.no_grad():
        result = model(img)[0]
    # print('Apply non-maximum supression to the results of Mega Detector... ')
    result_nms = non_max_suppression(prediction=result, conf_thres=DETECTION_THRESHOLD) # applying non-maximum supression logic to the results
    normalization = torch.tensor(img_array.shape)[[1, 0, 1, 0]]  # normalization gain whwh
    # extracting the feature from the object detected
    for detection in result_nms:
        if len(detection):
            detection[:, :4] = scale_coords(img.shape[2:], detection[:, :4], img_array.shape).round()
            for *xyxy, conf, cls in reversed(detection):
                # normalized center-x, center-y, width and height
                xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / normalization).view(-1).tolist()
                api_box = ct_utils.convert_yolo_to_xywh(xywh)
                conf = ct_utils.truncate_float(conf.tolist(), precision=3)
                # MegaDetector output format's categories start at 1, but this model's start at 0
                cls = int(cls.tolist())
                labels.append(DEFAULT_DETECTOR_LABEL_MAP[cls])
                scores.append(conf)
                bbox.append(ct_utils.truncate_float_array(api_box, precision=4))
                tot_detections.append({ 'category': cls, 'conf': conf, 'bbox': ct_utils.truncate_float_array(api_box, precision=4)})
                max_conf = max(max_conf, conf)
    final_output['max_detection_conf'] = max_conf
    final_output['detections'] = tot_detections
    if final_output['detections']:
        # extracting input filename
        # img_info = (os.path.basename(input_image_path)).split('.')
        # img_name = '.'.join(img_info)
        # building output file path to save an image
        # output_image_path = os.path.join(output_category_dir_path, f'{img_name}')
        # print(final_output)
        if final_output['detections'][0]['category'] == 0:
            # print('Animal detected')
            output_image_path = os.path.join(output_category_dir_path, os.path.basename(input_image_path))
            try:
                shutil.copy(input_image_path, output_image_path)
            except shutil.Error as ser:
                print(ser)

In [8]:
def create_directory(dir_path):
    """ Function to create a directory, if does not exists """
    if not os.path.isdir(dir_path):
        os.makedirs(dir_path)
        print(f'{os.path.basename(dir_path)}: {dir_path} directory created')

In [ ]:
from tqdm import tqdm
input_dir_list = os.listdir(input_dir_path)
for category in input_dir_list:
    # print(category)
    category_dir_path = os.path.join(input_dir_path, category)
    if os.path.isdir(category_dir_path):
        category_dir_list = os.listdir(category_dir_path)
        output_category_dir_path = os.path.join(output_dir_path, category)
        create_directory(output_category_dir_path)
        for img_file in tqdm(category_dir_list):
            img_file_path = os.path.join(category_dir_path, img_file)
            MegaDetector_Filter(img_file_path, output_category_dir_path)
        break
torch.cuda.empty_cache()

<h3>Ecology & Heritage Partners - Data Filtering</h3>

In [9]:
from tqdm import tqdm
input_dir_path = os.path.join(r"I:\ASDC - II")
input_dir_list = os.listdir(input_dir_path)
for survey_id in input_dir_list:
    # print(survey_id)
    survey_dir_path = os.path.join(input_dir_path, survey_id)
    if os.path.isdir(survey_dir_path):
        survey_dir_list = os.listdir(survey_dir_path)
        for site_id in survey_dir_list:
            # print(site_id)
            site_id_path = os.path.join(survey_dir_path, site_id)
            if os.path.isdir(site_id_path):
                # print(site_id_path)
                site_id_list = os.listdir(site_id_path)
                output_dir_path = os.path.join(f"{survey_dir_path}_Filter", site_id)
                create_directory(output_dir_path)
                for img_file in tqdm(site_id_list):
                    img_file_path = os.path.join(site_id_path, img_file)
                    MegaDetector_Filter(img_file_path, output_dir_path)
                torch.cuda.empty_cache()

20190402-20190527: I:\ASDC - II\11887_Filter\20190402-20190527 directory created


100%|████████████████████████████████████████████████████████████████████████████| 251/251 [00:10<00:00, 24.39it/s]


20190403-20190527: I:\ASDC - II\11887_Filter\20190403-20190527 directory created


100%|████████████████████████████████████████████████████████████████████████████| 300/300 [00:11<00:00, 26.17it/s]


20190403-20190528: I:\ASDC - II\11887_Filter\20190403-20190528 directory created


...

100%|████████████████████████████████████████████████████████████████████████████| 275/275 [00:20<00:00, 13.74it/s]


Site 3b west: I:\ASDC - II\5523_Filter\Site 3b west directory created


100%|████████████████████████████████████████████████████████████████████████████| 107/107 [00:07<00:00, 13.97it/s]


Site 3e east: I:\ASDC - II\5523_Filter\Site 3e east directory created


100%|████████████████████████████████████████████████████████████████████████████| 310/310 [00:22<00:00, 13.59it/s]
